In [1]:
!pip install -qU langchain sentence_transformers groq
!pip install -qU langchain-huggingface langchain-community
!pip install -qU faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 48.0 MB/s eta 0:00:00


In [2]:
from collections import Counter
from groq import Groq
from tqdm import tqdm
import numpy as np
import json
import faiss
import json
import pandas as pd

from google.colab import drive, files

drive.mount('/content/drive', force_remount=True)

#API_KEY = "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"
API_KEY = "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"
#API_KEY = "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"
client = Groq(
    api_key=API_KEY,
)

model_llm = "llama3-70b-8192"

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/IA024/ProjetoFinal

'Cópia de Plano do Projeto Final - RAG IRPF 2024 - Semana 2.gslides'
'Cópia de Plano do Projeto Final - RAG IRPF 2024 - Semana 3.gslides'
 df
'Documento sem título.gdoc'
 embedding_faiis.ipynb
 Extraçao_de_Normas.ipynb
 faiss
'frequencia_normas_padronizadas (4).xlsx'
'frequencia_normas_padronizadas - link - ok.xlsx'
'frequencia_normas_padronizadas - link.xlsx'
 json
'memrando links.docx'
'normas_extraidas (30).xlsx'
'Plano do Projeto Final - RAG.gslides'
'Plano do Projeto Final - RAG IRPF 2024 - Semana 2 (1).pptx'
'Plano do Projeto Final - RAG IRPF 2024 - Semana 3.gslides'
 prompts_tests.ipynb
 scraping_lei_json.ipynb
 scraping_solucoes_de_consulta_rfb.ipynb
 SCRAPING_VER_convert_txt.ipynb


In [4]:
json_normas_path = '/content/drive/MyDrive/IA024/ProjetoFinal/json/normas_full.json'

with open(json_normas_path, 'r') as file:
    normas_json = json.load(file)

normas = list(normas_json.values())
#normas[:3]

In [5]:
json_acordaos_path = '/content/drive/MyDrive/IA024/ProjetoFinal/json/acordaos.json'

with open(json_acordaos_path, 'r') as file:
    acordaos_json = json.load(file)

acordaos = list(acordaos_json.values())
#acordaos[:3]

In [6]:
normas_acordaos_json = {**normas_json, **acordaos_json}

In [7]:
csv_path = '/content/drive/MyDrive/IA024/ProjetoFinal/df/df_questions_answers.xlsx'

data_csv = pd.read_excel(csv_path)
data_csv.head()

,question_number,question_summary,question_text,answer,answer_cleaned,references,linked_questions
0,1,OBRIGATORIEDADE,Quem está obrigado a apresentar a Declaração d...,['Está obrigada a apresentar a Declaração de A...,['Está obrigada a apresentar a Declaração de A...,"Lei nº 9.250, de 26 de dezembro de 1995, art. ...",NaN
1,2,PESSOA FÍSICA DESOBRIGADA,Pessoa física desobrigada pode apresentar a De...,"['Sim. A pessoa física, ainda que desobrigada,...","['Sim. A pessoa física, ainda que desobrigada,...","Instrução Normativa RFB nº 2.178, de 5 de març...","['001', '174']"
2,3,TITULAR OU SÓCIO DE EMPRESA,Contribuinte que é titular ou sócio de empresa...,"['Não, a menos que se enquadre nas hipóteses p...","['Não, a menos que se enquadre nas hipóteses p...","Instrução Normativa RFB nº 2.178, de 5 de març...",['001']
3,4,QUADRO SOCIETÁRIO OU ASSOCIADO DE COOPERATIVA,"Contribuinte, que participou de quadro societá...","['Não, a menos que esteja obrigado a declarar ...","['Não, a menos que esteja obrigado a declarar ...","Instrução Normativa RFB nº 2.178, de 5 de març...",['001']
4,5,DE DECLARAR,Contribuinte que constou como responsável pera...,['Esse contribuinte está obrigado a declarar c...,['Esse contribuinte está obrigado a declarar c...,"Instrução Normativa RFB nº 2.178, de 5 de març...",['001']


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

delimiters = ["\nCAPÍTULO", "\nCapítulo", "\nSeção", "\nArt. ", "\n§", "\nI -", "\nII -", "\nIII -", "\nIV -", "\nV -", "\nVI -", "\nVII -", "\nVIII -", "\nIX -", "\nX -","\n"]

text_splitter = RecursiveCharacterTextSplitter(
    separators=delimiters,
    chunk_size=1000,  # Tamanho máximo do chunk em caracteres
    chunk_overlap=100  # Sobreposição entre chunks para contexto
)

chunks_normas = []
chunks_normas_acordaos = []
normas_acordaos = normas+acordaos

# Percorre cada item no JSON
for filename, doc in normas_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o documento em chunks
    chunks = text_splitter.split_text(doc)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        # Formata cada chunk com o nome do arquivo e o conteúdo
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas.append(chunk_with_name)

# Percorre cada item no JSON
for filename, doc in normas_acordaos_json.items():
    # Remove a extensão ".txt" do nome do arquivo
    doc_name = filename.replace(".txt", "")

    # Divide o documento em chunks
    chunks = text_splitter.split_text(doc)

    # Adiciona o nome do documento a cada chunk
    for chunk in chunks:
        # Formata cada chunk com o nome do arquivo e o conteúdo
        chunk_with_name = f"{doc_name}: {chunk}"
        chunks_normas_acordaos.append(chunk_with_name)

In [9]:
print(f'Total chunked normas: {len(chunks_normas)}')
print(f'Total chunked normas+acordaos: {len(chunks_normas_acordaos)}')

Total chunked normas: 25894
Total chunked normas+acordaos: 62743


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")
model = AutoModel.from_pretrained("stjiris/bert-large-portuguese-cased-legal-tsdae")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, 

In [14]:
import statistics

use_acordaos = True

if use_acordaos:
  chunks = chunks_normas_acordaos
else:
  chunks = chunks_normas


def contar_tokens(chunk):
    tokens = tokenizer(chunk, return_tensors="pt", truncation=False)  # Tokeniza o texto sem truncar
    num_tokens = len(tokens['input_ids'][0])  # Conta o número de tokens
    return num_tokens

# Calcule o número de tokens para cada chunk
num_tokens_por_chunk = [contar_tokens(chunk) for chunk in chunks]

# Cálculo das estatísticas
max_tokens = max(num_tokens_por_chunk)
min_tokens = min(num_tokens_por_chunk)
media_tokens = sum(num_tokens_por_chunk) / len(num_tokens_por_chunk)
mediana_tokens = statistics.median(num_tokens_por_chunk)

# Exibindo os resultados
print(f"Número máximo de tokens: {max_tokens}")
print(f"Número mínimo de tokens: {min_tokens}")
print(f"Média de tokens: {media_tokens}")
print(f"Mediana de tokens: {mediana_tokens}")

Número máximo de tokens: 966
Número mínimo de tokens: 11
Média de tokens: 209.2191160766938
Mediana de tokens: 227


In [15]:
def get_legal_bert_embeddings_batch(texts):
    # Tokeniza o batch de textos e converte para tensores
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move os inputs e o modelo para o dispositivo correto (CPU ou GPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = {key: value.to(device) for key, value in inputs.items()}
    model.to(device)

    # Desativa o cálculo de gradiente para otimizar a memória
    with torch.no_grad():
        outputs = model(**inputs)

    # Usa a média da última camada oculta como embedding para cada texto no batch
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Média de todos os tokens para representar o texto
    return embeddings.cpu().numpy()  # Converte para numpy para facilitar o uso

# Configura o tamanho do batch
batch_size = 8

# Gera embeddings para todos os chunks usando batches
embeddings = []
for i in tqdm(range(0, len(chunks), batch_size), desc="Gerando embeddings"):
    batch = chunks[i:i+batch_size]
    batch_embeddings = get_legal_bert_embeddings_batch(batch)
    embeddings.extend(batch_embeddings)

Gerando embeddings: 100%|██████████| 7843/7843 [59:52<00:00,  2.18it/s]


In [16]:
embeddings = np.array(embeddings).astype('float32')  # Certifique-se de que os embeddings são float32

# Cria um índice FAISS
d = embeddings.shape[1]  # Dimensão dos embeddings
index = faiss.IndexFlatL2(d)  # Usando o índice L2 (distância euclidiana)
index.add(embeddings)  # Adiciona os embeddings ao índice

faiss_index_file = 'faiss_index_acordaos_v3.index'  # Nome do arquivo onde o índice será salvo
faiss.write_index(index, faiss_index_file)  # Salva o índice

files.download(faiss_index_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>